In [13]:
from sympy import *
init_printing()

In [14]:
class Compartment(Function):
    nargs = 1

    def __str__(self):
        return f'[{self.args[0]}]'

    def _sympystr(self, printer=None):
        return f'[{self.args[0]}]'

    def _latex(self, printer=None):
        return '\\left[' + printer.doprint(self.args[0]) + '\\right]'

class ContentChange(Function):
    nargs = 2

    def __str__(self):
        return f'{self.args}'

    def _sympystr(self, printer=None):
        return f'{self.args}'

    def _latex(self, printer=None):
        return printer.doprint(self.args)


class Context:

    def __init__(self, numSpecies: int):
        self.D = numSpecies
        self.gamma = IndexedBase('\gamma', integer=True, shape=self.D)

    def compartment(self, *args):
        return Compartment(*args)

    def change(self, *args):
        return ContentChange(*args)

    def __str__(self):
        return f'Context({self.D})'


    def doit(self, transition):
        DM = Function('\Delta\mathcal{M}', real=True)
        mchange = Eq(DM(self.gamma), self.deltaM(transition))
        display(mchange)


    def deltaM(self, transition):
        return self.__deltaMExpr(transition.rhs) - self.__deltaMExpr(transition.lhs)


    def __deltaMExpr(self, expr):
        if expr.func == Compartment:
            content = expr.args[ 0 ]
            species = self.__getContentPerSpecies(content);
            return self.__getGamma(species)
        elif expr.func == EmptySet:
            return 0
        elif expr.func == Integer:
            return expr
        elif expr.func == Add:
            return Add(*[self.__deltaMExpr(i) for i in expr.args])
        elif expr.func == Mul:
            return Mul(*[self.__deltaMExpr(i) for i in expr.args])
        else:
            raise TypeError( "Unexpected expression " + str(expr))


    def __getContentPerSpecies(self, content):
        """Get an array of scalars representing compartment content for species 0..D"""

        if content.func == Add:
            xs = [self.__getContentPerSpecies(arg) for arg in content.args]
            return [Add(*x) for x in zip(*xs)]
        elif content.func == Mul:
            xs = [self.__getContentPerSpecies(arg) for arg in content.args]
            return [Mul(*x) for x in zip(*xs)]
        elif content.func == IndexedBase:
            return [content[i] for i in range(self.D)]
        elif content.func == ContentChange:
            return [content.args[i] for i in range(self.D)]
        elif content.func == Integer:
            return [content] * self.D
        else:
            raise TypeError( "Unexpected expression " + str(content))


    def __getGamma(self, content):
        return Mul(*[content[i] ** self.gamma[i] for i in range(self.D)])




class Transition(Basic):
    def __init__(self, lhs, rhs):
        self.lhs = lhs
        self.rhs = rhs

    def __str__(self):
        return f'{self.lhs} ---> {self.rhs})'

    def _latex(self, printer=None):
        # Always use printer.doprint() otherwise nested expressions won't
        # work. See the example of ModOpWrong.
        l = printer.doprint(self.lhs)
        r = printer.doprint(self.rhs)
        return l + '\longrightarrow{}' + r


In [15]:
C = Context(2)

X = IndexedBase('X', integer=True, shape=C.D)
Y = IndexedBase('Y', integer=True, shape=C.D)
Exit = Transition(C.compartment(X), EmptySet())
display(Exit)
C.doit(Exit)

[X] ---> EmptySet())

In [16]:
Coagulation = Transition(C.compartment(Y) + C.compartment(X), C.compartment(X+Y))
display(Coagulation)
C.doit(Coagulation)

[X] + [Y] ---> [X + Y])

In [17]:
ff(X[1],2)

In [18]:
factorial(3)